In [ ]:
import torch
import torch.nn as nn
import numpy as np
import cv2
import matplotlib.pyplot as plt
from ultralytics import YOLO
from tqdm import tqdm
from sort import Sort
import pickle, os

In [ ]:
# Model Definition remains the same
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', weights='ResNet50_Weights.DEFAULT')
        self.model.fc = nn.Identity()
        
    def forward(self, x):
        return self.model(x)

model = Net()
yolo = YOLO("yolov8n.pt")

In [ ]:
video_paths = ['aic19-track1-mtmc-train/train/S01/c001/vdo.avi', 
               'aic19-track1-mtmc-train/train/S01/c002/vdo.avi', 
               'aic19-track1-mtmc-train/train/S01/c003/vdo.avi',
               'aic19-track1-mtmc-train/train/S01/c004/vdo.avi',
               'aic19-track1-mtmc-train/train/S01/c005/vdo.avi',
               'aic19-track1-mtmc-train/train/S03/c010/vdo.avi',
               'aic19-track1-mtmc-train/train/S03/c011/vdo.avi',
               'aic19-track1-mtmc-train/train/S03/c012/vdo.avi',
               'aic19-track1-mtmc-train/train/S03/c013/vdo.avi',
               'aic19-track1-mtmc-train/train/S03/c014/vdo.avi',
               'aic19-track1-mtmc-train/train/S03/c015/vdo10.avi',
               'aic19-track1-mtmc-train/train/S04/c016/vdo.avi',
               'aic19-track1-mtmc-train/train/S04/c017/vdo.avi',
               'aic19-track1-mtmc-train/train/S04/c018/vdo.avi',
               'aic19-track1-mtmc-train/train/S04/c019/vdo.avi',
               'aic19-track1-mtmc-train/train/S04/c020/vdo.avi',
               'aic19-track1-mtmc-train/train/S04/c021/vdo.avi',
               'aic19-track1-mtmc-train/train/S04/c022/vdo.avi',
               'aic19-track1-mtmc-train/train/S04/c023/vdo.avi',
               'aic19-track1-mtmc-train/train/S04/c024/vdo.avi',
               'aic19-track1-mtmc-train/train/S04/c025/vdo.avi',
               'aic19-track1-mtmc-train/train/S04/c026/vdo.avi',
               'aic19-track1-mtmc-train/train/S04/c027/vdo.avi',
               'aic19-track1-mtmc-train/train/S04/c028/vdo.avi',
               'aic19-track1-mtmc-train/train/S04/c029/vdo.avi',
               'aic19-track1-mtmc-train/train/S04/c030/vdo.avi',
               'aic19-track1-mtmc-train/train/S04/c031/vdo.avi',
               'aic19-track1-mtmc-train/train/S04/c032/vdo.avi',
               'aic19-track1-mtmc-train/train/S04/c033/vdo.avi',
               'aic19-track1-mtmc-train/train/S04/c034/vdo.avi',
               'aic19-track1-mtmc-train/train/S04/c035/vdo.avi',
               'aic19-track1-mtmc-train/train/S04/c036/vdo.avi',
               'aic19-track1-mtmc-train/train/S04/c037/vdo.avi',
               'aic19-track1-mtmc-train/train/S04/c038/vdo.avi',
               'aic19-track1-mtmc-train/train/S04/c039/vdo.avi',
               'aic19-track1-mtmc-train/train/S04/c040/vdo.avi']

In [ ]:
def upscale_fps(input_path, output_path, input_fps=8, target_fps=10):
    """
    This function upscales the FPS of a video by interpolating frames using OpenCV and NumPy.
    
    Parameters:
    input_path (str): The path to the input video file.
    output_path (str): The path where the output video will be saved.
    input_fps (int): The original FPS of the video.
    target_fps (int): The target FPS to upscale the video to.
    """
    import cv2
    import numpy as np
    
    # Capture video from input path
    cap = cv2.VideoCapture(input_path)
    if not cap.isOpened():
        return "Failed to open video file."
    
    # Get video properties
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(output_path, fourcc, target_fps, (width, height))
    
    prev_frame = None
    success, frame = cap.read()
    while success:
        out.write(frame)  # Write the original frame
        
        # If there's a previous frame, generate interpolated frames
        if prev_frame is not None:
            # Generate interpolated frames
            for _ in range((target_fps // input_fps) - 1):
                interp_frame = cv2.addWeighted(prev_frame, 0.5, frame, 0.5, 0)
                out.write(interp_frame)
                
        prev_frame = frame
        success, frame = cap.read()
        
    # Release everything if job is finished
    cap.release()
    out.release()
    return "Video FPS upscaled successfully."

# upscale_fps('aic19-track1-mtmc-train/train/S03/c015/vdo.avi', 'aic19-track1-mtmc-train/train/S03/c015/vdo10.avi')

In [ ]:
def create_detections(video_path, yolo):
    """
    Given a video, create detections for each frame in the video and store bounding boxes in a text file
    """
    cap = cv2.VideoCapture(video_path)

    # Create a text file to store the bounding boxes
    f = open(f"detections/{video_path.split('/')[-2].split('.')[0]}.txt", "w")

    frame_num = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Get the detections for the frame
        detections = yolo(frame, verbose=False)

        # Write the bounding boxes to the text file
        for box in detections[0].boxes:
            if box.cls == 2:
                x1, y1, x2, y2 = box.xyxy[0].int().tolist()
                f.write(f"{frame_num},{x1},{y1},{x2},{y2}\n")

        frame_num += 1

    f.close()
    cap.release()

# for video in tqdm(video_paths):
#     create_detections(video, yolo)

In [ ]:
# Import necessary libraries
import numpy as np

# Initialization
camera_trackers = {camera_id: Sort() for camera_id in camera_ids}  # SORT tracker for each camera
global_tracks = []

def update_global_tracks(tracked_detections, camera_id):
    """
    Update global tracks with new detections from a specific camera.
    This function attempts to match new detections to existing global tracks
    and creates new global tracks for unmatched detections.
    """
    # Placeholder for matching logic between tracked_detections and global_tracks
    # This function should use the siamese_nn to compute similarities
    # and match detections across cameras, updating global_tracks accordingly.

def track_across_cameras(detections):
    """
    Main function to handle tracking across multiple cameras.
    """
    # Step 1: Update tracks within each camera using SORT
    local_tracks = {}
    for camera_id, dets in detections.items():
        tracked_detections = camera_trackers[camera_id].update(np.array(dets))
        local_tracks[camera_id] = tracked_detections

    # Step 2: Update global tracks with detections from all cameras
    for camera_id, tracked_detections in local_tracks.items():
        update_global_tracks(tracked_detections, camera_id)

    # Step 3 (Optional): Handle cross-camera tracking logic
    # This could involve additional steps to refine track matching across cameras,
    # especially if tracks move from the view of one camera to another.

track_across_cameras(detections)
